In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree

In [20]:
testa= pd.read_csv('tcd-ml-1920-group-income-train.csv', delimiter = ',')

C:\Users\eoinb\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
testa['Housing Situation'] = testa['Housing Situation'].replace('nA', np.nan)
testa['Work Experience in Current Job [years]'] = testa['Work Experience in Current Job [years]'].replace('#NUM!', np.nan)
testa['Yearly Income in addition to Salary (e.g. Rental Income)'] = testa['Yearly Income in addition to Salary (e.g. Rental Income)'].replace({' EUR':''}, regex = True)

testa['Work Experience in Current Job [years]'] = testa['Work Experience in Current Job [years]'].astype(float)
testa['Yearly Income in addition to Salary (e.g. Rental Income)'] = testa['Yearly Income in addition to Salary (e.g. Rental Income)'].astype(float)

In [22]:
for col in testa.columns:
        if testa[col].dtype == object:
            testa[col] = testa[col].fillna(testa[col].mode()[0])
        else:
            testa[col] = testa[col].fillna(testa[col].mean())

In [23]:
labels = testa['Total Yearly Income [EUR]']

In [24]:
encoder = ce.TargetEncoder(cols=['Hair Color', 'Housing Situation','Satisfation with employer' ,'Country', 'Profession', 'University Degree', 'Gender'])
encoder.fit(testa, labels)
testa = encoder.transform(testa)

In [25]:
train_features = testa.drop(['Total Yearly Income [EUR]'], axis = 1)

In [26]:
train_features= train_features.drop('Instance', axis=1)
scaler = MinMaxScaler()
scaler.fit(train_features, labels)
train_features = scaler.transform(train_features)

In [27]:
X_test, X_val, y_test, y_val = train_test_split(train_features, labels, test_size = 0.15, random_state = 42)

In [32]:
trn_data = lgb.Dataset(X_test, label=y_test)
val_data = lgb.Dataset(X_val, label=y_val)
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.03,
                        n_estimators=2000)
gbm.fit(X_test, y_test,
        eval_set=[(X_val, y_val)],
        eval_metric='l1',
        early_stopping_rounds=5)

[1]	valid_0's l1: 70911.1	valid_0's l2: 1.20717e+10
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 68990.8	valid_0's l2: 1.15008e+10
[3]	valid_0's l1: 67130.7	valid_0's l2: 1.09627e+10
[4]	valid_0's l1: 65340.8	valid_0's l2: 1.0454e+10
[5]	valid_0's l1: 63610.2	valid_0's l2: 9.9757e+09
[6]	valid_0's l1: 61932.8	valid_0's l2: 9.52206e+09
[7]	valid_0's l1: 60307.9	valid_0's l2: 9.09429e+09
[8]	valid_0's l1: 58754.1	valid_0's l2: 8.69273e+09
[9]	valid_0's l1: 57243.7	valid_0's l2: 8.30987e+09
[10]	valid_0's l1: 55781.5	valid_0's l2: 7.95014e+09
[11]	valid_0's l1: 54368.4	valid_0's l2: 7.60705e+09
[12]	valid_0's l1: 52996.8	valid_0's l2: 7.28671e+09
[13]	valid_0's l1: 51672	valid_0's l2: 6.98366e+09
[14]	valid_0's l1: 50393.6	valid_0's l2: 6.69383e+09
[15]	valid_0's l1: 49159.7	valid_0's l2: 6.42149e+09
[16]	valid_0's l1: 47962.3	valid_0's l2: 6.16305e+09
[17]	valid_0's l1: 46803.5	valid_0's l2: 5.91806e+09
[18]	valid_0's l1: 45699.4	valid_0's l2: 5.68674e+09

[156]	valid_0's l1: 12313.5	valid_0's l2: 8.4376e+08
[157]	valid_0's l1: 12282	valid_0's l2: 8.4029e+08
[158]	valid_0's l1: 12249.8	valid_0's l2: 8.37301e+08
[159]	valid_0's l1: 12207.9	valid_0's l2: 8.33308e+08
[160]	valid_0's l1: 12177	valid_0's l2: 8.30516e+08
[161]	valid_0's l1: 12152.2	valid_0's l2: 8.28287e+08
[162]	valid_0's l1: 12114.9	valid_0's l2: 8.24812e+08
[163]	valid_0's l1: 12097.7	valid_0's l2: 8.22681e+08
[164]	valid_0's l1: 12077.1	valid_0's l2: 8.20552e+08
[165]	valid_0's l1: 12055.3	valid_0's l2: 8.18209e+08
[166]	valid_0's l1: 12011.4	valid_0's l2: 8.14718e+08
[167]	valid_0's l1: 11994.9	valid_0's l2: 8.12562e+08
[168]	valid_0's l1: 11979.1	valid_0's l2: 8.10705e+08
[169]	valid_0's l1: 11963	valid_0's l2: 8.08457e+08
[170]	valid_0's l1: 11935.3	valid_0's l2: 8.05947e+08
[171]	valid_0's l1: 11914.7	valid_0's l2: 8.0384e+08
[172]	valid_0's l1: 11898.3	valid_0's l2: 8.01739e+08
[173]	valid_0's l1: 11866.2	valid_0's l2: 7.98524e+08
[174]	valid_0's l1: 11845.4	valid_0's

[311]	valid_0's l1: 10648.4	valid_0's l2: 6.51304e+08
[312]	valid_0's l1: 10644.7	valid_0's l2: 6.50647e+08
[313]	valid_0's l1: 10643.6	valid_0's l2: 6.50303e+08
[314]	valid_0's l1: 10638.2	valid_0's l2: 6.49665e+08
[315]	valid_0's l1: 10634.1	valid_0's l2: 6.49322e+08
[316]	valid_0's l1: 10631.2	valid_0's l2: 6.48885e+08
[317]	valid_0's l1: 10628.9	valid_0's l2: 6.48582e+08
[318]	valid_0's l1: 10626.8	valid_0's l2: 6.48197e+08
[319]	valid_0's l1: 10623.9	valid_0's l2: 6.47673e+08
[320]	valid_0's l1: 10618.2	valid_0's l2: 6.47099e+08
[321]	valid_0's l1: 10614.7	valid_0's l2: 6.46509e+08
[322]	valid_0's l1: 10611	valid_0's l2: 6.46083e+08
[323]	valid_0's l1: 10609.9	valid_0's l2: 6.45718e+08
[324]	valid_0's l1: 10608.5	valid_0's l2: 6.45303e+08
[325]	valid_0's l1: 10605.3	valid_0's l2: 6.44795e+08
[326]	valid_0's l1: 10602.1	valid_0's l2: 6.44327e+08
[327]	valid_0's l1: 10598.3	valid_0's l2: 6.43954e+08
[328]	valid_0's l1: 10596.9	valid_0's l2: 6.43702e+08
[329]	valid_0's l1: 10593	vali

[464]	valid_0's l1: 10281.2	valid_0's l2: 6.03904e+08
[465]	valid_0's l1: 10280.7	valid_0's l2: 6.03651e+08
[466]	valid_0's l1: 10279.5	valid_0's l2: 6.03517e+08
[467]	valid_0's l1: 10277.2	valid_0's l2: 6.03311e+08
[468]	valid_0's l1: 10276.9	valid_0's l2: 6.03257e+08
[469]	valid_0's l1: 10276	valid_0's l2: 6.03086e+08
[470]	valid_0's l1: 10275.2	valid_0's l2: 6.02988e+08
[471]	valid_0's l1: 10274.7	valid_0's l2: 6.02919e+08
[472]	valid_0's l1: 10274.5	valid_0's l2: 6.02879e+08
[473]	valid_0's l1: 10273.6	valid_0's l2: 6.02657e+08
[474]	valid_0's l1: 10271.3	valid_0's l2: 6.02529e+08
[475]	valid_0's l1: 10270.5	valid_0's l2: 6.02388e+08
[476]	valid_0's l1: 10267.1	valid_0's l2: 6.02143e+08
[477]	valid_0's l1: 10266.8	valid_0's l2: 6.02098e+08
[478]	valid_0's l1: 10265.9	valid_0's l2: 6.0199e+08
[479]	valid_0's l1: 10265.3	valid_0's l2: 6.01864e+08
[480]	valid_0's l1: 10264.6	valid_0's l2: 6.0169e+08
[481]	valid_0's l1: 10263.7	valid_0's l2: 6.01574e+08
[482]	valid_0's l1: 10262.9	vali

[620]	valid_0's l1: 10098.4	valid_0's l2: 5.83917e+08
[621]	valid_0's l1: 10098.2	valid_0's l2: 5.83903e+08
[622]	valid_0's l1: 10097.8	valid_0's l2: 5.83868e+08
[623]	valid_0's l1: 10097.4	valid_0's l2: 5.8381e+08
[624]	valid_0's l1: 10096.8	valid_0's l2: 5.83692e+08
[625]	valid_0's l1: 10096.6	valid_0's l2: 5.83675e+08
[626]	valid_0's l1: 10095.8	valid_0's l2: 5.83598e+08
[627]	valid_0's l1: 10095.5	valid_0's l2: 5.83559e+08
[628]	valid_0's l1: 10094.3	valid_0's l2: 5.83504e+08
[629]	valid_0's l1: 10094.3	valid_0's l2: 5.83503e+08
[630]	valid_0's l1: 10093.8	valid_0's l2: 5.8339e+08
[631]	valid_0's l1: 10092.7	valid_0's l2: 5.83179e+08
[632]	valid_0's l1: 10092.3	valid_0's l2: 5.83054e+08
[633]	valid_0's l1: 10091.3	valid_0's l2: 5.82963e+08
[634]	valid_0's l1: 10090.2	valid_0's l2: 5.82829e+08
[635]	valid_0's l1: 10090	valid_0's l2: 5.82812e+08
[636]	valid_0's l1: 10089.9	valid_0's l2: 5.82784e+08
[637]	valid_0's l1: 10089.6	valid_0's l2: 5.82744e+08
[638]	valid_0's l1: 10088.5	vali

[776]	valid_0's l1: 9965.18	valid_0's l2: 5.69361e+08
[777]	valid_0's l1: 9965.07	valid_0's l2: 5.69237e+08
[778]	valid_0's l1: 9964.26	valid_0's l2: 5.69164e+08
[779]	valid_0's l1: 9962.98	valid_0's l2: 5.69063e+08
[780]	valid_0's l1: 9961.41	valid_0's l2: 5.68985e+08
[781]	valid_0's l1: 9961.24	valid_0's l2: 5.68936e+08
[782]	valid_0's l1: 9961.19	valid_0's l2: 5.68935e+08
[783]	valid_0's l1: 9961.07	valid_0's l2: 5.68797e+08
[784]	valid_0's l1: 9960.58	valid_0's l2: 5.68692e+08
[785]	valid_0's l1: 9959.95	valid_0's l2: 5.6861e+08
[786]	valid_0's l1: 9959.96	valid_0's l2: 5.68608e+08
[787]	valid_0's l1: 9958.98	valid_0's l2: 5.68567e+08
[788]	valid_0's l1: 9958.76	valid_0's l2: 5.68538e+08
[789]	valid_0's l1: 9957.9	valid_0's l2: 5.68496e+08
[790]	valid_0's l1: 9957.51	valid_0's l2: 5.68436e+08
[791]	valid_0's l1: 9957.66	valid_0's l2: 5.68434e+08
[792]	valid_0's l1: 9956.2	valid_0's l2: 5.68236e+08
[793]	valid_0's l1: 9955.81	valid_0's l2: 5.68183e+08
[794]	valid_0's l1: 9953.88	val

[930]	valid_0's l1: 9871.6	valid_0's l2: 5.58827e+08
[931]	valid_0's l1: 9871.29	valid_0's l2: 5.58778e+08
[932]	valid_0's l1: 9871.13	valid_0's l2: 5.58764e+08
[933]	valid_0's l1: 9870.22	valid_0's l2: 5.58637e+08
[934]	valid_0's l1: 9869.01	valid_0's l2: 5.58561e+08
[935]	valid_0's l1: 9867.13	valid_0's l2: 5.58351e+08
[936]	valid_0's l1: 9866.88	valid_0's l2: 5.58325e+08
[937]	valid_0's l1: 9866.53	valid_0's l2: 5.58277e+08
[938]	valid_0's l1: 9864.71	valid_0's l2: 5.58177e+08
[939]	valid_0's l1: 9863.01	valid_0's l2: 5.58037e+08
[940]	valid_0's l1: 9861.16	valid_0's l2: 5.57837e+08
[941]	valid_0's l1: 9860.94	valid_0's l2: 5.57805e+08
[942]	valid_0's l1: 9860.58	valid_0's l2: 5.57759e+08
[943]	valid_0's l1: 9860.3	valid_0's l2: 5.57734e+08
[944]	valid_0's l1: 9860.31	valid_0's l2: 5.57738e+08
[945]	valid_0's l1: 9859.93	valid_0's l2: 5.57657e+08
[946]	valid_0's l1: 9859.77	valid_0's l2: 5.5764e+08
[947]	valid_0's l1: 9859.76	valid_0's l2: 5.57635e+08
[948]	valid_0's l1: 9859.42	val

[1085]	valid_0's l1: 9778.77	valid_0's l2: 5.49723e+08
[1086]	valid_0's l1: 9778.01	valid_0's l2: 5.49628e+08
[1087]	valid_0's l1: 9776.84	valid_0's l2: 5.49577e+08
[1088]	valid_0's l1: 9776.54	valid_0's l2: 5.49559e+08
[1089]	valid_0's l1: 9776.17	valid_0's l2: 5.49525e+08
[1090]	valid_0's l1: 9775.55	valid_0's l2: 5.49485e+08
[1091]	valid_0's l1: 9775.27	valid_0's l2: 5.49462e+08
[1092]	valid_0's l1: 9774.75	valid_0's l2: 5.49411e+08
[1093]	valid_0's l1: 9774.63	valid_0's l2: 5.49383e+08
[1094]	valid_0's l1: 9773.37	valid_0's l2: 5.49277e+08
[1095]	valid_0's l1: 9773.33	valid_0's l2: 5.49277e+08
[1096]	valid_0's l1: 9772.98	valid_0's l2: 5.49255e+08
[1097]	valid_0's l1: 9772.77	valid_0's l2: 5.49209e+08
[1098]	valid_0's l1: 9772	valid_0's l2: 5.49093e+08
[1099]	valid_0's l1: 9771.96	valid_0's l2: 5.49101e+08
[1100]	valid_0's l1: 9771.95	valid_0's l2: 5.49084e+08
[1101]	valid_0's l1: 9771.95	valid_0's l2: 5.49105e+08
[1102]	valid_0's l1: 9771.45	valid_0's l2: 5.49029e+08
[1103]	valid_

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.03, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [29]:
#submission

#aa = CatBoostRegressorlgb()
#aa.fit(X_test, y_test);

In [30]:
income_pred_a = gbm.predict(X_val)

In [31]:
mean_absolute_error(y_val, income_pred_a)

9745.57490534551